In [1]:
import sys
sys.path.append("../../../h2o-py/build/main")

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import h2o
from h2o.estimators import H2ODecisionTreeEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()

versionFromGradle='3.46.0',projectVersion='3.46.0.99999',branch='valenad-test-test',lastCommitHash='d7daf66d53941dbd75639a0ec6ffe1b2910de62a',gitDescribe='jenkins-3.46.0.2-1-gd7daf66d53',compiledOn='2024-05-24 14:11:07',compiledBy='potte'
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.2+8-LTS-86, mixed mode, sharing)
  Starting server from D:\skola\dip\h2o-3\build\h2o.jar
  Ice root: C:\Users\potte\AppData\Local\Temp\tmpoxyi2zqi
  JVM stdout: C:\Users\potte\AppData\Local\Temp\tmpoxyi2zqi\h2o_potte_started_from_python.out
  JVM stderr: C:\Users\potte\AppData\Local\Temp\tmpoxyi2zqi\h2o_potte_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.99999
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_potte_5fj9rr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,23.94 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [4]:
titanic = h2o.import_file('train-prep.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
titanic_train = h2o.import_file('train-split.csv')
titanic_valid = h2o.import_file('valid-split.csv')
target_variable = 'Survived'
train_variables =['Age','Sex','Pclass']

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
valid = titanic_valid
train = titanic_train

In [7]:
titanic_valid[titanic_valid['Survived'] == '0'].shape

(0, 11)

In [8]:
titanic_valid[titanic_valid['Survived'] == '1'].shape

(0, 11)

In [9]:
train

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,nan,7.25,S
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,nan,7.925,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,S
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,C
13,0,3,"Saundercock, Mr. William Henry",male,20,0,0,nan,8.05,S
16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55,0,0,248706,16,S
17,0,3,"Rice, Master. Eugene",male,2,4,1,382652,29.125,Q
21,0,2,"Fynney, Mr. Joseph J",male,35,0,0,239865,26,S


In [10]:
train[train["Survived"] == '0'].shape

(0, 11)

In [11]:
train[train["Survived"] == '1'].shape

(0, 11)

In [12]:
valid

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,nan,71.2833,C
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,S
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,nan,16.7,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,S
14,0,3,"Andersson, Mr. Anders Johan",male,39,1,5,347082,31.275,S
15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14,0,0,350406,7.8542,S
19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)",female,31,1,0,345763,18,S
24,1,1,"Sloper, Mr. William Thompson",male,28,0,0,113788,35.5,S
25,0,3,"Palsson, Miss. Torborg Danira",female,8,3,1,349909,21.075,S


In [13]:
def make_categorical(dataset):
    dataset[target_variable] = dataset[target_variable].asfactor()
    dataset["Sex"] = dataset["Sex"].asfactor()

In [14]:
make_categorical(train)
make_categorical(valid)

In [15]:
sdt_h2o = H2ODecisionTreeEstimator(model_id="decision_tree.hex", max_depth=5,seed =15468)
sdt_h2o.train(y=target_variable,x = train_variables,training_frame=train)

dt Model Build progress: |███████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODecisionTreeEstimator : Decision Tree
Model Key: decision_tree.hex

No summary for this model

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [16]:
predictions = sdt_h2o.predict(valid)
predictions

dt prediction progress: |████████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.0508475,0.949153
0,0.936709,0.0632911
0,0.647059,0.352941
1,0.445946,0.554054
1,0.0508475,0.949153
0,0.936709,0.0632911
1,0.445946,0.554054
1,0.445946,0.554054
0,0.605634,0.394366
1,0.445946,0.554054


In [17]:
make_categorical(titanic)

In [18]:
titanic_drf = H2ORandomForestEstimator(ntrees=100,max_depth=5,min_rows=10,calibrate_model=True,calibration_frame=valid,
                                    binomial_double_trees=True,seed = 15624)
titanic_drf.train(x=train_variables,y=target_variable,training_frame=train,validation_frame=valid)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1716912638373_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                200                         36533                  1            5            4.62          2             16            9.925

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.13004047042876143
RMSE: 0.36061124556613794
LogLoss: 0.4206329064468465
Mean Per-Class Error: 0.17888368462138954
AUC: 0.8765807962529273
AUCPR: 0.8219966714614717
Gini: 0.7531615925058546

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3401750550553558
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      238  67   0.2197   (67.0/305.0)
1      29   181  0.1381   (29.0/210.0)
Total  267  248  0.1864   (96.0/515.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.340175     0.790393  215
max f2                       0.249049     0.851064  249
max f0point5                 0.63293      0.822281  114
max accuracy                 0.520626     0.825243  165
max precision                0.939237     1         0
max recall                   0.0732335    1         399
max specificity              0.939237     1         0
max absolute_mcc             0.436626     0.635632  184
max min_per_class_accuracy   0.399862     0.814286  192
max mean_per_class_accuracy  0.340175     0.821116  215
max tns                      0.939237     305       0
max fns                      0.939237     209       0
max fps                      0.0742477    305       398
max tps                      0.0732335    210       399
max tnr                      0.939237     1         0
max fnr                      0.939237     0.995238  0
max fpr                      0.0742477    1         398
max tpr                      0.0732335    1         399

Gains/Lift Table: Avg response rate: 40,78 %, avg score: 40,61 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0116505                   0.916166           1.63492   1.63492            0.666667         0.924959   0.666667                    0.924959            0.0190476       0.0190476                  63.4921   63.4921            0.0124902
2        0.0213592                   0.906855           2.45238   2.00649            1                0.91028    0.818182                    0.918287            0.0238095       0.0428571                  145.238   100.649            0.0362998
3        0.031068                    0.902104           2.45238   2.14583            1                0.905602   0.875                       0.914323            0.0238095       0.0666667                  145.238   114.583            0.0601093
4        0.0407767                   0.893649           1.9619    2.10204            0.8              0.898551   0.857143                    0.910568            0.0190476       0.0857143                  96.1905   110.204            0.0758782
5        0.0504854                   0.885825           2.45238   2.16941            1                0.890174   0.884615                    0.906646            0.023

In [19]:
predicts = titanic_drf.predict(valid)
predicts

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1,cal_p0,cal_p1
1,0.0890001,0.911,0.152152,0.847848
0,0.880413,0.119587,0.833427,0.166573
0,0.499612,0.500388,0.502205,0.497795
1,0.317517,0.682483,0.319321,0.680679
1,0.169374,0.830626,0.201033,0.798967
0,0.890133,0.109867,0.839024,0.160976
0,0.426426,0.573574,0.425819,0.574181
0,0.450136,0.549864,0.450357,0.549643
0,0.622786,0.377214,0.62873,0.37127
1,0.363223,0.636777,0.362461,0.637539


In [20]:
titanic_drf.model_performance(valid = True)

ModelMetricsBinomial: drf
** Reported on validation data. **

MSE: 0.17689040854280194
RMSE: 0.4205834144884959
LogLoss: 0.5397612495072264
Mean Per-Class Error: 0.24337427278603752
AUC: 0.7577030812324931
AUCPR: 0.7775371621625464
Gini: 0.5154061624649862

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5963106487923832
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      116  3    0.0252   (3.0/119.0)
1      36   42   0.4615   (36.0/78.0)
Total  152  45   0.198    (39.0/197.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.596311     0.682927  31
max f2                       0.0900093    0.766208  115
max f0point5                 0.596311     0.813953  31
max accuracy                 0.596311     0.80203   31
max precision                0.916937     1         0
max recall                   0.0900093    1         115
max specificity              0.916937     1         0
max absolute_mcc             0.596311     0.597892  31
max min_per_class_accuracy   0.35049      0.692308  62
max mean_per_class_accuracy  0.596311     0.756626  31
max tns                      0.916937     119       0
max fns                      0.916937     75        0
max fps                      0.0900093    119       115
max tps                      0.0900093    78        115
max tnr                      0.916937     1         0
max fnr                      0.916937     0.961538  0
max fpr                      0.0900093    1         115
max tpr                      0.0900093    1         115

Gains/Lift Table: Avg response rate: 39,59 %, avg score: 38,49 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0152284                   0.916937           2.52564   2.52564            1                0.916937   1                           0.916937            0.0384615       0.0384615                  152.564   152.564            0.0384615
2        0.0253807                   0.911              2.52564   2.52564            1                0.911      1                           0.914562            0.025641        0.0641026                  152.564   152.564            0.0641026
3        0.0304569                   0.90533            2.52564   2.52564            1                0.910862   1                           0.913945            0.0128205       0.0769231                  152.564   152.564            0.0769231
4        0.0406091                   0.885148           2.52564   2.52564            1                0.89556    1                           0.909349            0.025641        0.102564                   152.564   152.564            0.102564
5        0.0507614                   0.884816           2.52564   2.52564            1                0.884882   1                           0.904456            0.025641        0.128205                   152.564   152.564            0.128205
6        0.106599                    0.870809           2.52564   2.52564            1                0.875789   1                           0.88944             0.141026        0.269231                   152.564   152.564            0.269231
7        0.152284                    0.833408           2.24501   2.44145            0.888889         0.84954    0.966667                    0.87747             0.102564        0.371795                   124.501   144.145            0.363392
8        0.203046                    0.652303  

In [21]:
gbm_params1 = {'learn_rate': [0.01, 0.1],
               'ntrees' : [10,50,100],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}

In [22]:
gbm_gridtitanic = H2OGridSearch(model=H2OGradientBoostingEstimator,grid_id='gbm_gridtitanic',hyper_params=gbm_params1)

gbm_gridtitanic.train(x=train_variables, y=target_variable,training_frame=train,validation_frame=valid,seed=1)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,col_sample_rate,learn_rate,max_depth,ntrees,sample_rate,model_ids,logloss
,1.0,0.01,9.0,100.0,1.0,gbm_gridtitanic_model_105,0.5205040
,1.0,0.1,9.0,10.0,1.0,gbm_gridtitanic_model_72,0.5207325
,1.0,0.1,9.0,10.0,0.8,gbm_gridtitanic_model_18,0.5230928
,1.0,0.01,9.0,100.0,0.8,gbm_gridtitanic_model_51,0.5233986
,1.0,0.1,5.0,10.0,0.8,gbm_gridtitanic_model_12,0.5236181
,1.0,0.01,5.0,100.0,0.8,gbm_gridtitanic_model_45,0.5240269
,1.0,0.1,5.0,10.0,1.0,gbm_gridtitanic_model_66,0.5266201
,1.0,0.01,5.0,100.0,1.0,gbm_gridtitanic_model_99,0.5272014
,1.0,0.1,3.0,10.0,0.8,gbm_gridtitanic_model_6,0.5290455
,1.0,0.01,3.0,100.0,0.8,gbm_gridtitanic_model_39,0.5315531


In [23]:
gbm_gridperf1 = gbm_gridtitanic.get_grid(sort_by='auc', decreasing=True)
gbm_gridperf1

,col_sample_rate,learn_rate,max_depth,ntrees,sample_rate,model_ids,auc
,1.0,0.1,5.0,100.0,0.8,gbm_gridtitanic_model_48,0.7969726
,1.0,0.1,9.0,50.0,0.8,gbm_gridtitanic_model_36,0.7887847
,1.0,0.1,9.0,100.0,1.0,gbm_gridtitanic_model_108,0.7884615
,0.2,0.1,9.0,100.0,1.0,gbm_gridtitanic_model_106,0.7871148
,0.5,0.1,9.0,100.0,1.0,gbm_gridtitanic_model_107,0.7871148
,0.2,0.1,9.0,100.0,0.8,gbm_gridtitanic_model_52,0.7859836
,0.5,0.1,9.0,100.0,0.8,gbm_gridtitanic_model_53,0.7859836
,1.0,0.1,5.0,50.0,1.0,gbm_gridtitanic_model_84,0.7854988
,1.0,0.1,9.0,50.0,1.0,gbm_gridtitanic_model_90,0.7854449
,1.0,0.01,9.0,100.0,1.0,gbm_gridtitanic_model_105,0.7851217


In [24]:
best_gbm1 = gbm_gridperf1.models[0]
best_gbm1

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: gbm_gridtitanic_model_48


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         21859                  5            5            5             6             21            12.82

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.08460965792120398
RMSE: 0.2908773932797184
LogLoss: 0.28215591396951334
Mean Per-Class Error: 0.11717408274785324
AUC: 0.9500858704137394
AUCPR: 0.9407439128128429
Gini: 0.9001717408274788

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4406406353845877
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      280  25   0.082    (25.0/305.0)
1      32   178  0.1524   (32.0/210.0)
Total  312  203  0.1107   (57.0/515.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.440641     0.861985  107
max f2                       0.15314      0.881057  156
max f0point5                 0.744749     0.898533  82
max accuracy                 0.557524     0.891262  100
max precision                0.994079     1         0
max recall                   0.0645275    1         191
max specificity              0.994079     1         0
max absolute_mcc             0.557524     0.774056  100
max min_per_class_accuracy   0.360414     0.87541   118
max mean_per_class_accuracy  0.440641     0.882826  107
max tns                      0.994079     305       0
max fns                      0.994079     206       0
max fps                      0.0139618    305       228
max tps                      0.0645275    210       191
max tnr                      0.994079     1         0
max fnr                      0.994079     0.980952  0
max fpr                      0.0139618    1         228
max tpr                      0.0645275    1         191

Gains/Lift Table: Avg response rate: 40,78 %, avg score: 40,48 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0135922                   0.989107           2.45238   2.45238            1                0.992211   1                           0.992211            0.0333333       0.0333333                  145.238   145.238            0.0333333
2        0.0252427                   0.98796            2.45238   2.45238            1                0.988212   1                           0.990365            0.0285714       0.0619048                  145.238   145.238            0.0619048
3        0.0349515                   0.986864           2.45238   2.45238            1                0.986905   1                           0.989404            0.0238095       0.0857143                  145.238   145.238            0.0857143
4        0.0407767                   0.985096           2.45238   2.45238            1                0.986486   1                           0.988987            0.0142857       0.1                        145.238   145.238            0.1
5        0.0524272                   0.976905           2.45238   2.45238            1                0.979897   1                           0.986967            0.0285714       

In [25]:
#titanic_gbm = H2OGradientBoostingEstimator(nfolds=5,seed=1111,keep_cross_validation_predictions = True)
#titanic_gbm.train(x=train_variables,y=target_variable, training_frame=titanic)

In [26]:
predict = best_gbm1.predict(valid)
predict

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.0346144,0.965386
0,0.931811,0.0681889
1,0.736917,0.263083
1,0.122918,0.877082
1,0.115536,0.884464
0,0.922314,0.0776859
1,0.167585,0.832415
1,0.426485,0.573515
1,0.623046,0.376954
1,0.817557,0.182443


In [27]:
valid = valid.as_data_frame()

D:\skola\dip\h2o-3\_valenad-notebooks\machine-learning-experiments\H2O\../../../h2o-py/build/main\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [28]:
predicts = predicts['predict'].as_data_frame()

In [29]:
predict = predict['predict'].as_data_frame()

In [30]:
valid = valid['Survived'] 

In [31]:
score=accuracy_score(valid, predict)
score

0.6954314720812182

In [32]:
score=accuracy_score(valid, predicts)
score

0.8020304568527918